<a href="https://colab.research.google.com/github/monika-the-unicorn/Choice-assay/blob/main/1_DLC_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook uses DLC to analyse a folder of new videos in the VideoDrive from a round of beecam data collection

## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

Install dependencies (will take a few mins)


In [ ]:
!apt update && apt install cuda-11-8
!pip install "deeplabcut[tf]"

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,227 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,527 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,560 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [45.2 kB]
Get:13 https://r2u.stat.illino

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)**

### Link to Google Drive

In [ ]:
# Run this cell and follow the authorization instructions:

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Set up project variables:
ProjectFolderName = "Beecam_v4-Anna-2024-06-25"
VideoType = "mp4"

In [ ]:
import deeplabcut

Loading DLC 2.3.10...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Check DLC loaded:

In [ ]:
deeplabcut.__version__

'2.3.10'

Link to DLC project configuration file:

In [ ]:
# Creates a path variable that links to google drive copy
# (No need to edit this)
path_config_file = "/content/drive/My Drive/" + ProjectFolderName + "/config.yaml"
path_config_file

'/content/drive/My Drive/Beecam_v4-Anna-2024-06-25/config.yaml'

***

###The following code takes DLC output files i.e. csvs showing where each of the body parts was in each frame and how confident in this estimation the program was; and extracts info about when parts of the proboscis were present


*   Provide a big folder containing all the videos collected in the round
*   Create a master sheet with absolute time in the left column, an RPi variable, a tube variable, and a behaviour column (for drinking/prob out)

In order to populate this with information about when and where bees were showing proboscis activity, we then need to loop through all the video folders


*   Loop through video folders (all footage taken by each raspberry pi). For each folder:
  * Save the raspberry pi number
  * Save start time from the last part of the folder name
  * Open the Videos folder

  * For each video in the video folder:
    i) Save start time from file name as a value
    ii) Save tube side as a value
    iii) Run positional data extraction and convert to csv
    iv) Subset rows in the csv with proboscis middle and/or end present p > 0.6 for at least x consecutive frames
    v) Convert frame number to absolute time
    vi) Fill in behaviour column as drinking or searching by prob tube +- end
    vii) Append time, RPi, tube, and behaviour to the master sheet

* Once all proboscis activity has been recorded, normalise the time by subtracting the relative starting time for each RPi from the time values


#ANALYSE NEW VIDEOS

### Set experiment and round to analyse

In [ ]:
# ACTION REQUIRED: Enter round number (for saving file) and change project data folder if necessary
ROUND = 22
PROJECT_DATA_FOLDER = "/content/drive/My Drive/BeecamData/Other drugs"
path_export = PROJECT_DATA_FOLDER + "/DLC_output/masterdf_round" + str(ROUND) + ".csv"

print(path_export)

/content/drive/My Drive/BeecamData/Other drugs/DLC_output/masterdf_round22.csv


In [ ]:
# ACTION REQUIRED: DEFINE FOLDER CONTAINING VIDEOS FROM THE ROUND (all videos to analyse)
folder_to_analyse = (
    "Other drugs November 2024/Round 22"  # File inside the VideoDrive folder to analyse (i.e. SOURCE_DIR)
)
# Don't change these:
folder_to_analyse_path = (
    "/content/drive/My Drive/VideoDrive/" + folder_to_analyse
)  # i.e. WORKING_DIRECTORY = "G:/Shared drives/VideoDrive/AMDR Run2"
DEST_DIR = folder_to_analyse_path + "/" + "Processed_videos"
path_config_file = "/content/drive/My Drive/" + ProjectFolderName + "/config.yaml"

### Then determine which videos have already been analysed

Videos in the round:

In [ ]:
import os


def list_all_videos(source_dir):
    # List to hold the paths of mp4 files
    all_videos = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        # Iterate over all files in the current root
        for file in files:
            if file.endswith(".mp4"):
                if file not in all_videos:
                    video_path = os.path.join(root, file)
                    video_name = video_path.split("/Round 22/")[1]
                    all_videos.append(video_name)

    print(len(all_videos), "videos in this round")
    return all_videos


all_videos = list_all_videos(folder_to_analyse_path)

694 videos in this round


In [ ]:
def list_processed_videos(source_dir, search_string="DLC_resnet50"):
    """
    Walks through the source directory and its subdirectories to find files containing the search_name in their names.

    :param source_dir: The root directory to start searching from.
    :param search_name: The substring to look for in file names. Default is "DLC_resnet50".
    :return: A list of file paths that contain the search_name in their names.
    """
    # List to hold the paths of matching files
    processed_videos = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        # Iterate over all files in the current root
        for file_name in files:
            if search_string in file_name:
                processed_file_path = os.path.join(root, file_name)
                # Extract the video name only from it
                video_name = processed_file_path.split("/Round 22/")[1].split("DLC")[0] + ".mp4"
                # Append the video name to the list if it is not a duplicate
                if video_name not in processed_videos:
                    processed_videos.append(video_name)

    print(len(processed_videos), "videos already processed and added to list")
    return processed_videos


processed_videos = list_processed_videos(folder_to_analyse_path)

0 videos already processed and added to list


In [ ]:
# Get files to process


def get_unprocessed_videos(all_videos, processed_videos):
    processed_set = set(processed_videos)
    unprocessed_videos = [video for video in all_videos if video not in processed_set]

    return unprocessed_videos


unprocessed_videos = get_unprocessed_videos(all_videos, processed_videos)
print(len(unprocessed_videos), "videos to process")

694 videos to process


## Process unprocessed videos

### Just video processing (makes csvs in the same file/folder as original videos; doesn't perform behavioural data extraction on them)

In [ ]:
RPicounter = 0
Videocounter = 0

folder_list = os.listdir(folder_to_analyse_path)  # Lists all the RPi folders in the round

for folder in folder_list:
    RPicounter += 1  # Add 1 to the RPi counter when open new RPi folder
    Videocounter = 0  # Reset video counter to 0 when open new RPi folder
    # Make a list of video files that the pi recorded, and feedback its length
    filelist = [
        file
        for file in os.listdir(os.path.join(folder_to_analyse_path, folder, "Videos"))
        if file.endswith(".mp4")
    ]  # List all the videos in the round for that RPi
    Videototal = len(filelist)  # Count the number of videos in the round for that RPi
    print("RPi", RPicounter, "recorded", Videototal, "videos")

    # Extract data from them if haven't already been processed, with information about how many have been processed
    for video in filelist:
        Videocounter += 1
        print(
            "**********Processing video",
            Videocounter,
            "of",
            Videototal,
            "********** from RPi number",
            RPicounter,
            "in list **********",
        )
        videofile_path = os.path.join(
            folder_to_analyse_path, folder, "Videos", video
        )  # Create local variable with whole video file path
        video_check_path = os.path.join(
            folder, "Videos", video
        )  # Create local variable with RPi/round/video file path to be checked against those that have already been processed in the list
        # Skip if already processed
        if video_check_path in processed_videos:
            print(video_check_path, "already analysed")
        else:
            deeplabcut.analyze_videos(
                path_config_file, videofile_path, videotype=VideoType, save_as_csv=True
            )  # Save output to csv
            print("Video", Videocounter, "analysed")

RPi 1 recorded 22 videos
**********Processing video 1 of 22 ********** from RPi number 1 in list **********
Using snapshot-186000 for model /content/drive/My Drive/Beecam_v4-Anna-2024-06-25/dlc-models/iteration-0/Beecam_v4Jun25-trainset95shuffle1


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Starting to analyze %  /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi9_03-12-2024_11-03-34/Videos/left_04-12-2024_09-02-39.mp4
Loading  /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi9_03-12-2024_11-03-34/Videos/left_04-12-2024_09-02-39.mp4
Duration of video [s]:  289.8 , recorded with  5.0 fps!
Overall # of frames:  1449  found with (before cropping) frame dimensions:  436 436
Starting to extract posture


100%|██████████| 1449/1449 [00:24<00:00, 60.04it/s]
/usr/local/lib/python3.10/dist-packages/deeplabcut/utils/auxiliaryfunctions.py:402: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")


Saving results in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi9_03-12-2024_11-03-34/Videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Video 1 analysed
**********Processing video 2 of 22 ********** from RPi number 1 in list **********
Using snapshot-186000 for model /content/drive/My Drive/Beecam_v4-Anna-2024-06-25/dlc-models/iteration-0/Beecam_v4Jun25-trainset95shuffle1


...

100%|██████████| 49/49 [00:01<00:00, 38.55it/s]
/usr/local/lib/python3.10/dist-packages/deeplabcut/utils/auxiliaryfunctions.py:402: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")
/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Saving results in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi5_02-12-2024_19-17-37/Videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Video 3 analysed
**********Processing video 4 of 4 ********** from RPi number 10 in list **********
Using snapshot-186000 for model /content/drive/My Drive/Beecam_v4-Anna-2024-06-25/dlc-models/iteration-0/Beecam_v4Jun25-trainset95shuffle1
Starting to analyze %  /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi5_02-12-2024_19-17-37/Videos/left_02-12-2024_19-17-44.mp4
Loading  /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi5_02-12-2024_19-17-37/Videos/left_02-12-2024_19-17-44.m

100%|██████████| 65/65 [00:01<00:00, 44.56it/s]

Saving results in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 22/Beecam_RPi5_02-12-2024_19-17-37/Videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Video 4 analysed



/usr/local/lib/python3.10/dist-packages/deeplabcut/utils/auxiliaryfunctions.py:402: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  DataMachine.to_hdf(dataname, "df_with_missing", format="table", mode="w")
